In [1]:
import psycopg2

def getConnection():

    # Connect to postgres with a copy of the MIMIC-III database
    con = psycopg2.connect(
        dbname='mimic4',
        user='postgres',
        host='superbugai.erc.monash.edu',
        port=5434,
        password='mysecretpassword'
        )

    return con


In [2]:
import pandas as pd


con = getConnection()
sql = 'select * from omop_test_20220817.data_matrix'
dataDf = pd.read_sql_query(sql=sql, con=con)

/tmp/ipykernel_1034724/45782066.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataDf = pd.read_sql_query(sql=sql, con=con)


In [3]:
dataDf.anchor_time = dataDf.anchor_time.apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S'))
dataDf.death_datetime = dataDf.death_datetime.apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S'))

In [6]:
dataDf.shape

(2483, 155)

In [5]:
dataDf.drop(dataDf[dataDf['death_datetime'] < (dataDf['anchor_time'] + pd.Timedelta(hours=24))].index)

,person_id,age,gender,ethnicity_WHITE,ethnicity_BLACK,ethnicity_UNKNOWN,ethnicity_OTHER,ethnicity_HISPANIC,ethnicity_ASIAN,ethnicity_UNABLE_TO_OBTAIN,...,hemoglobin_last,platelet_count_last,urea_nitrogen_last,glucose_serum_last,bicarbonate_last,potassium_serum_last,anion_gap_last,leukocytes_blood_manual_last,hematocrit_last,aptt_last
0,-2145189891,31.0,0,0,0,1,0,0,0,0,...,-0.936966,0.099286,-0.876253,-0.349127,-0.095777,-0.212789,-0.006983,-0.329169,-0.898441,-0.487866
1,-2144283250,52.0,0,0,0,0,0,0,0,1,...,-0.410143,0.114766,-0.471009,-0.834255,-0.989249,-1.475719,0.748908,1.089534,-0.283250,-0.394958
2,-2143572246,38.0,1,0,1,0,0,0,0,0,...,0.485458,0.563671,-0.786198,0.010226,1.021063,-0.393207,-0.762873,2.880357,0.313847,-0.698457
3,-2143338028,46.0,0,1,0,0,0,0,0,0,...,-0.620872,1.136413,-0.606090,0.621127,-0.542513,-0.754044,1.252835,0.124351,-0.771784,1.710954
4,-2142538649,70.0,1,1,0,0,0,0,0,0,...,1.170329,-0.457976,0.519588,0.207871,-0.765881,0.508886,0.748908,0.449955,1.037601,0.069581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2478,2137799179,62.0,1,1,0,0,0,0,0,0,...,-0.410143,-0.651469,-0.471009,0.495353,0.127591,0.328467,-1.518764,-0.445457,-0.645127,0.354499
2479,2138356045,88.0,0,1,0,0,0,0,0,0,...,-1.095013,0.060588,-0.696144,0.153968,-1.212617,1.591398,-0.006983,-0.282655,-0.844159,-0.593161
2480,2142310139,52.0,1,1,0,0,0,0,0,0,...,2.750800,-0.744346,-0.741171,-0.636610,1.691167,-1.114882,-0.762873,-0.131481,2.448921,0.713743
2481,2144178407,85.0,0,1,0,0,0,0,0,0,...,-0.989649,-0.295441,-0.020737,-0.924093,-1.212617,-0.573626,-0.006983,0.054578,-0.572751,-0.686069


In [7]:
dataDf['target'] = (dataDf['death_datetime'] > (dataDf['anchor_time'] + pd.Timedelta(days=0))) & (dataDf['death_datetime'] < (dataDf['anchor_time'] + pd.Timedelta(days=7)))


In [10]:
dataDf.target.value_counts()

target
False    2117
True      366
Name: count, dtype: int64